In [ ]:
import glob
import os
import pandas as pd

In [ ]:
data_dir = os.path.join(".", "data", "crsp")
csv_files = sorted(glob.glob(os.path.join(data_dir, "*.csv")))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {data_dir}")

df = pd.read_csv(csv_files[0])
#display(df.head())

df.columns = [col.strip().lower() for col in df.columns]
#display(df.columns.tolist())

In [ ]:
df.columns = [col.strip().lower() for col in df.columns]
keep_columns = [
    "ticker",
    "permco",
    "dlycaldt",
    "dlycap",
    "dlyret",
    "dlyvol",
    "shrout",
    "sprtrn",
]
missing = [col for col in keep_columns if col not in df.columns]
if missing:
    raise KeyError(f"Missing columns in CSV: {missing}")

In [ ]:
df = df[keep_columns].rename(columns={"dlycaldt": "timestamp"})
df = df[
    [
        "timestamp",
        "permco",
        "ticker",
        "dlyret",
        "dlycap",
        "dlyvol",
        "shrout",
        "sprtrn",
    ]
]

In [ ]:
permco_values = sorted(df["permco"].dropna().unique().tolist())
display(f"permco: {permco_values}")
#display(df.columns.tolist())
display(f"rows: {df.shape[0]}, columns: {df.shape[1]}")
#display(df.head())

In [ ]:
def first_valid(series: pd.Series) -> object:
    first_idx = series.first_valid_index()
    if first_idx is None:
        return pd.NA
    return series.loc[first_idx]

def format_ticker(value: object) -> str:
    if pd.isna(value):
        return "unknown"
    ticker = str(value).strip()
    return ticker if ticker else "unknown"


def format_permco(value: object) -> str:
    if pd.isna(value):
        return "unknown"
    if isinstance(value, float) and value.is_integer():
        return str(int(value))
    return str(value)


In [ ]:
df_sorted = df.sort_values(["permco", "timestamp"], kind="mergesort")
last_ticker_by_permco = df_sorted.groupby("permco", sort=False)["ticker"].last()
permco_to_ticker = {
    permco: format_ticker(ticker)
    for permco, ticker in last_ticker_by_permco.items()
}
value_columns = ["dlyret", "dlycap", "dlyvol", "shrout", "sprtrn"]

wide = df.pivot_table(
    index="timestamp",
    columns="permco",
    values=value_columns,
    aggfunc=first_valid,
)
wide = wide.swaplevel(0, 1, axis=1)
ordered_columns = [
    (permco, value) for permco in permco_values for value in value_columns
]
wide = wide.reindex(columns=pd.MultiIndex.from_tuples(ordered_columns))
wide.columns = [
    f"{format_permco(permco)}_{permco_to_ticker.get(permco, 'unknown')}_{value}"
    for permco, value in wide.columns
]
wide = wide.reset_index()
#display(permco_values)
#display(wide.columns.tolist())
display(f"rows: {wide.shape[0]}, columns: {wide.shape[1]}")
display(wide.head())


In [ ]:
#display(wide.columns.tolist())